# BNClassifier Tutorial

This notebook illustrates the workflow of `BNClassifier`. We present the workflow on a short 
 study using a PSBN model of the MAPK pathway. The details are described in `case-study-mapk.pdf` (in the `tutorial` folder).

We give a step-by-step guide on how to replicate the results and observations. In the first part, we show how to run the classification and explore its inputs/outputs by executing the Python cells of this notebook. In the second part, we illustrate the GUI capabilities through animations, screenshots and comments. Please refer to the enclosed `Manual` for more information on the tool.

First, please read the installation instructions provided in the main README.

After everything is successfully set up, start executing the following cells, one by one, to run the classification process.

In [1]:
# load modules
import subprocess

from pathlib import Path
from biodivine_aeon import *

## Exploring the input

We first demonstrate what the input for the HCTL classification looks like. It is essentially a combination of:
- A partially specified Boolean network and corresponding influences.
- Required properties (HCTL formulae) which restrict the set of valid model interpretations.
- Classification properties (HCTL formulae) that are used to classify the valid models.

We can explore the PSBN model and its properties:

In [2]:
input_path = Path("case-study-mapk") / "mapk-annotated.aeon"
bn = BooleanNetwork.from_file(str(input_path))
print(bn)

BooleanNetwork(variables = 18, parameters = 0, regulations = 60)


In [3]:
model_string = input_path.read_text(encoding="utf-8")
annotations = ModelAnnotation.from_model_string(model_string)
required_properties = get_model_assertions(annotations)
classification_properties = get_model_properties(annotations)

for prop in required_properties:
    print("Required property:\t", prop)
print()
for (name, prop) in classification_properties:
    print(f"Classification property named `{name}`:\t", prop)

Required property:	 3{x}: @{x}: ((AG EF {x}) & (Apoptosis | Growth_Arrest | Proliferation))

Classification property named `p1`:	 V{x}: @{x}: ((AG EF {x}) => ((AX {x}) & Apoptosis))
Classification property named `p2`:	 3{x}: @{x}: (AG Growth_Arrest)
Classification property named `p3`:	 3{x}: @{x}: (AG Proliferation)
Classification property named `p4`:	 3{x}: @{x}: ((AG EF {x}) & Apoptosis & Proliferation)


The required properties are given as plain HCTL formulae (in our case, it is a single formula). On the other hand, classification properties also have names, which are used in the post-processing of the classification results (as we'll see later). The format of the input is further described in the tool's Manual).

To reiterate, these properties mean the following:
- `p1`: Every attractor state is a fixed point with Apoptosis active (the system always converges to programmed cell death; "healthy" behaviour).
- `p2`, `p3`: Growth Arrest (`p2`) or Proliferation (`p3`) remain active in some attractor (programmed cell death may not be achieved; possibly "problematic" behaviour).
- `p4`: Proliferation and Apoptosis are simultaneously active in an attractor state (biologically "unrealistic" behaviour).

## Running the classification engine

The classifier categorizes PSBN interpretations into classes based on the properties they satisfy. First, only the interpretations satisfying all required properties are computed. The set of remaining interpretations is decomposed into categories, where each class contains interpretations that (universally) satisfy the same set of properties.

It is fairly straightforward to run the classification from Jupyter notebooks. You provide a path to the input file with an annotated model and a path for the result bundle. Everything else is handled internally.

Note that this step might take a few minutes. **The runtime depends on your machine, but it should generally be around 1-10 minutes.**

In [4]:
output_zip = "classification-archive.zip"
run_hctl_classification(str(input_path), output_zip)

Loaded model and properties out of `case-study-mapk/mapk-annotated.aeon`.
Parsing formulae and generating symbolic representation...
Successfully parsed all 1 required property (assertion) and all 4 classification properties.
Successfully encoded model with 18 variables and 4 parameters.
Model admits 16 instances.
Evaluating required properties (this may take some time)...
Required properties successfully evaluated.
15 instances satisfy all required properties.
Evaluating classification properties (this may take some time)...
Classification properties successfully evaluated.
Generating classification mapping based on model-checking results...
Results saved to `classification-archive.zip`.


The result is a `zip-archive` with:
- A text report regarding the whole classification procedure.
- Raw BDD files encoding each non-empty class.
- The original annotated input model (to reconstruct the results if needed).

Before loading the output bundle into our interactive GUI explorer, we show how to directly obtain general information about the results. We can load the dictionary with all categories - a map of `string -> color set` pairs (colors representing individual interpretations). The string "key" represents a binary encoding of which properties are satisfied by the particular class (in this encoding, properties are ordered alphabetically). We can further analyse the results in Python, but that is out of the scope of this tutorial.

In [5]:
class_mapping, original_model_str = load_class_archive(output_zip)
print(class_mapping)

{'0100': ColorSet(cardinality=3), '0001': ColorSet(cardinality=3), '1100': ColorSet(cardinality=9)}


Here, for example:
- 9 colors satisfy properties `p1` and `p2` simulataneously.
- 3 colors satisfy only `p2`
- 3 colors satisfy only `p4`
- No color satisfies property `p3`

## Exploring the results in GUI

Let's analyze the results in the Decision Tree Explorer GUI. Before starting the GUI, you can watch a summary of all the steps (that we'll go through later) in the following GIF animation. Once it finishes (less than 1 minute), feel free to follow the step-by-step guide with commentary below it. Note that the animation loops.

![GIF-animation](images/Animation.gif)

To start the GUI, please run the following cell. It executes the provided HCTL explorer binary. You will be prompted to pick a file to open. Here, you should select `./tutorial/classification-archive.zip` (the output of the previous step).

In [6]:
subprocess.run(["../bin/hctl-explorer"])
# if the command above by chance does not work, you can also try the following version:
# subprocess.run(["hctl-explorer"])

You should see the following screen with simple navigation instructions:

![Initial layout](images/1.png)

The tree's root node with the label `Mixed outcomes (3 types)` should appear. By scrolling, you can zoom in/out. You can move the "camera" around by dragging anywhere in the empty space.

![Loaded-root](images/2_3.png)

Once you adjust the node's size and position, click on the node. On the left, a panel describing the node should appear (see the following image). In this panel, you can see: 

- The root node represents interpretations of three different classes. For each class ("behaviour"), we see which properties it corresponds to, how many interpretations (witnesses) it represents, and what percentage of total number of interpretations it is. Particularly, nine interpretations satisfy properties `p1` and `p2`, three satisfy only `p2`, and three only `p4` (as we have seen before).
- Property `p3` is `universally invalid` in the whole tree - no interpretation satisfies it.

![Root-panel](images/3.png)

The tool offers two ways to expand the node - automatically and manually. First, to try out the manual expanding, click on the `Make decision` button. Several possible `Decision Attributes` appear.

![Click-manual-expand](images/4.png)

Decision attributes can be sorted by several metrics. More on that in the Manual. Select the second item, `DNA_damage`.

![Click-decision-dna](images/5.png)

The tree should now expand - the root node becomes a `decision node` labelled by `DNA_damage`, and two new mixed nodes appear.

We refer to child nodes as "positive" (the one to which the green arrow points) and "negative" (the one to which the red arrow points). The positive child node represents interpretations where the parameter `DNA_damage` is set to `True`.
Note that the tree layout algorithm is not fully deterministic, and therefore the position of the branches may be switched on some machines.
Sometimes, the child nodes switch positions during tree expansion to allow for a better layout.

Let's focus on the negative `Mixed` child node (on a "red path" from the root). Select the node.

![Expanded-tree-v1](images/6.png)

On the left, a different panel should appear again. We see that this node now represents only two different classes. Thus, when `DNA_damage` is set to `False`, we can no longer reach a class where `p2` is satisfied and `p1` is not.

Click on the button `Auto expand (1 level)`, automatically choosing a decision attribute for this node and expanding it.

![Level1-negativeNode-panel](images/7.png)

The tree should again expand - the selected node becomes a `decision node` labelled with `TGFBR_stimulus`, and two new nodes appear. 

The new nodes are both leaf nodes. They each represent a subset of interpretations of a single class. To see more, click on the leaf node labelled by `p4 (3)`. From the node's label, we see that it represents three interpretations that satisfy a single classification property `p4`. 

![Expanded-tree-v2](images/8.png)

A different panel on the left should appear, as shown below. There, we see a set of `Necessary conditions` to reach the node. These correspond to the path from the root node. In particular, both `TGFBR_stimulus` and `DNA_damage` must be set to `False` (denoted by the red colour).

Properties satisfied in the node are also highlighted. In this case, all three BN instances represented by the node admit attractors where `Apoptosis` and `Proliferation` are active at the same time.

In a leaf node, you can generate `witness networks` (BN instances). Click on `Generate network(s)` button to export a particular BN instance. A `File save dialog` should appear.

![Witness-generating](images/9.png)

Now, let's automatically expand the rest of the tree. Click on the remaining `Mixed outcomes` node. In the panel that appears (see below), change the depth of `Automatic tree expansion` to 3 using the `Depth` toggle (there is a maximum of 3 remaining decision attributes).

![Auto-expand-toggle](images/10.png)

 Click on the button `Auto expand (3 levels)`.

![Auto-expand-3-levels](images/11.png)

The tree fully expands, as shown below (note that the position of some branches can be different, as discussed above).

![Fully-expanded-tree](images/12.png)

We can now explore the conditions leading to the validity of individual properties. In particular, we see that the property `p4` occurs if and only if both `DNA_damage` and `TGFBR_stimulus` are inactive. The decision tree also guides us towards conditions that ensure proper programmed cell death (`p1`). For example, we see that the presence of `TGFBR_stimulus` is a sufficient condition for `p1`.

We can also export the current tree in the `dot` format. Click on the `Export decision tree` button in the lower left corner. A `File save dialog` should appear.

![Export](images/13.png)

And this concludes the Tutorial.